In [66]:
import tensorflow as tf
import os
from pathlib import Path

In [67]:
train_path = Path() / 'aclImdb' / 'train'
test_path = Path() / 'aclImdb' / 'test'

In [68]:
unsupervised_path = train_path / 'unsup'
if os.path.exists(unsupervised_path):
    for file in os.listdir(unsupervised_path):
        file_path = unsupervised_path / file
        os.remove(file_path)
    os.rmdir(unsupervised_path)
    print('Unsupervised dataset removed')
else:
    print('Unsupervised dataset does not exist')    

Unsupervised dataset does not exist


In [69]:
train_ds = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=32,
    seed=42,
)

Found 25000 files belonging to 2 classes.


In [70]:
for i, label in enumerate(train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [71]:
(test_ds, val_ds) = tf.keras.utils.text_dataset_from_directory(
    test_path,
    batch_size=32,
    validation_split=0.6,
    subset='both',
    seed=42,
)

Found 25000 files belonging to 2 classes.
Using 10000 files for training.
Using 15000 files for validation.


In [72]:
for i, label in enumerate(test_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [73]:
vocab_size = 10_000
embedding_dim = 16

In [74]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [75]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [76]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15)

Epoch 1/15
782/782 [==============================] - 16s 19ms/step - loss: 0.6430 - accuracy: 0.5585 - val_loss: 0.5151 - val_accuracy: 0.7205
Epoch 2/15
782/782 [==============================] - 8s 10ms/step - loss: 0.3930 - accuracy: 0.8260 - val_loss: 0.3466 - val_accuracy: 0.8366
Epoch 3/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2914 - accuracy: 0.8810 - val_loss: 0.3131 - val_accuracy: 0.8513
Epoch 4/15
782/782 [==============================] - 8s 11ms/step - loss: 0.2480 - accuracy: 0.8988 - val_loss: 0.3155 - val_accuracy: 0.8385
Epoch 5/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2229 - accuracy: 0.9124 - val_loss: 0.2901 - val_accuracy: 0.8713
Epoch 6/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2019 - accuracy: 0.9212 - val_loss: 0.2862 - val_accuracy: 0.8805
Epoch 7/15
782/782 [==============================] - 8s 11ms/step - loss: 0.1845 - accuracy: 0.9290 - val_loss: 0.3047 - val_accuracy: 0.862

In [78]:
import tensorflow_datasets as tfds

In [79]:
dataset = tfds.load('imdb_reviews')
dataset

{'train': <_PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'test': <_PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'unsupervised': <_PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>}

In [80]:
train_ds = dataset['train']
full_test_ds = dataset['test']

# out of the test set we take 80% for validation and 20% for testing

In [81]:
len(full_test_ds)

25000

In [82]:
test_ratio = 0.2

test_len = int(len(full_test_ds) * test_ratio)

test_dataset = full_test_ds.take(test_len)
val_dataset = full_test_ds.skip(test_len)

In [83]:
len(test_dataset)

5000

In [84]:
len(val_dataset)

20000

In [86]:
model.fit(
    train_ds,
    validation_data=val_dataset,
    epochs=15)

Epoch 1/15


ValueError: in user code:

    File "C:\Users\farou\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\farou\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\farou\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\farou\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\farou\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\faro

In [87]:
train_ds

<_PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [108]:
for x in val_dataset:
    print(tuple(x.values))
    raise Exception()

TypeError: 'builtin_function_or_method' object is not iterable

In [106]:
for x in val_ds:
    print(x)
    raise Exception()

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'I don\'t know why all of the critics say this was a bad movie, this is one of my all-time favorite movies. Tommy (Chris Farley) and Richard (David Spade) are a great match. Tommy is the stupid one who spent seven years in college, finally passing with a D+. Richard is the smart kid who was hated by everyone in school. How can you not laugh when the two of them are singing "Superstar" by The Carpenters passionatly and all of a sudden the hood pops up? In all this is a great movie, I recommend it.',
       b'Do you ever wonder what is the worst movie ever made? Stop wondering. I\'m telling you, Michael is it!<br /><br />It is not "heartwarming," "entertaining," or "Travolta at his best." It just sucks. If I had kids, I would let them watch Deep Throat before Michael!<br /><br />A sold-out John Travolta, a washed-up and balding William Hurt, and an about to die any time now Jean Stapleton highlight this turd of a film.<br /><br />But

Exception: 